In [1]:
import librosa

### Observation

In [ ]:
from scipy.io import wavfile as wav
wave_sample_rate, wave_audio=wav.read(audio_file_path)

In [ ]:
wave_audio

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
plt.plot(wave_audio)

### Extract Features

We will be using Mel-Frequency Cepstral Coefficients(MFCC) from the audio samples. The MFCC summarizes the frequency distribution across the window size, so it is possible to analyze both the frequency and time characteristics of the sound. These audio representations will allow us to identify features for classification.

In [ ]:
mfccs=librosa.feature.mfcc(y=librosa_audio_data,sr=librosa_sample_rate,n_mfcc=40)
print(mfccs.shape)

In [ ]:
mfccs

In [ ]:
import pandas as pd
import os
import librosa
audio_dataset_path="urb"
metadata=pd.read_csv()
metadata.head()

In [ ]:
def features_extractor(file):
    audio,sample_rate=librosa.load(file_name,res_type="kaiser_fast")
    mfccs_features=librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
    mfccs_scaled_features=np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

In [ ]:
import numpy as np
from tqdm import tqdm
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row['fold'])+'/',str(row['slice_file_name']))
    final_class_labels=row['class']
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
X_train

In [ ]:
y

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

### Model Creation

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from sklearn import metrics

In [ ]:
num_lables.shape[1]

In [ ]:
Dense()

In [ ]:
# model=Sequential()
# model.add(Dense(100,input_shape=(40,)))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(200))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(100))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_labels))
# model.add(Activation('softmax'))

In [ ]:
model=Sequential([
    Dense(100,input_shapes=(40,),activation='relu'),
    Dropout(0,5),
    Dense(200,activation='relu'),
    Dropout(0.5),
    Dense(100,activation='relu'),
    Dropout(0.5),
    Dense(num_labels,activation='relu')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
num_epochs=100
num_batch_size=32
checkpointer=ModelCheckpoint(filepath='',verbose=1,save_best_only=True)
start=datetime.now()
model.fit(X_train,y_train,batch_size=batch_size,epochs=num_epochs,validation_data=(X_test,y_test),callbacks=[checkpointer])
duration=datetime.now()-start
print("Training completed in time: ",duration)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test, verbose=0)
print(test_accuracy[1])

In [ ]:
filename=""
prediction_feature=features_extractor(filename)
prediction_feature=prediction_feature.reshape(1,-1)
model.predict_classes(prediction_feature)

In [ ]:
prediction_feature.shape

In [ ]:
X_test[1]

In [ ]:
model.predict_classes(X_test)

### Testing Some Test Audio Data

Steps
* Preprocess the new audio data
* predict the classes
* Inverse Transform your Predicted Label

In [ ]:
filename=""
audio, sample_rate=librosa.load(file_name,res_type="kaiser_fast")
mfccs_features=librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
mfccs_scaled_features=np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class=labelencoder.inverse_transform(predicted_label)
prediction_class